In [3]:
from datasets import load_dataset

ds = load_dataset("matejklemen/clc_fce")

In [4]:
all_ds= [ds["train"], ds["test"], ds["validation"]]
print(all_ds[0].column_names)

['src_tokens', 'tgt_tokens', 'corrections']


In [ ]:
import re

import re

def detokenize(tokens):
    sentence = ""
    open_double_quote = True
    open_single_quote = True
    
    for token in tokens:
        if token == '"':
            if open_double_quote:
                sentence += ' "' if sentence and not sentence[-1].isspace() else '"'
                open_double_quote = False
            else:
                sentence = sentence.rstrip() + '" '
                open_double_quote = True
        elif token == "'":
            if open_single_quote:
                sentence += " '" if sentence and not sentence[-1].isspace() else "'"
                open_single_quote = False
            else:
                sentence = sentence.rstrip() + "' "
                open_single_quote = True
        elif token in {".", ",", "!", "?", ":", ";", "-"}:
            if sentence and sentence[-1] == ' ':
                sentence = sentence[:-1]
            sentence += token
        elif token in {")", "]", "}"}:
            sentence = sentence.rstrip() + token
        elif token in {"(", "[", "{"}:
            sentence += token
        else:
            no_space_tokens = ["'s", "n't", "'S", "'m", "'ll", "'ve", "'d", "'re", "--", ".-"]
            prev_no_space_tokens = ["'", "$", "-", '"', '"', ' ', '£']
            if sentence and sentence[-1] not in prev_no_space_tokens and token not in no_space_tokens:
                sentence += " "
            sentence += token
    return sentence

# Example usage
tokens = [
    "I", "am", "writing", "in", "order", "to", "express", "my", 
    "disappointment", "about", "your", "musical", "show", "\"", 
    "Over", "the", "Rainbow", "\"", "."
]
sentence = detokenize(tokens)
print(sentence)

I am writing in order to express my disappointment about your musical show "Over the Rainbow".
/Users/frankli/Desktop/Yale Classes/Classes 2024/Ling/final/ling-final-project/get_data_scripts


In [6]:
import pandas as pd
import nltk
from nltk.tokenize import TreebankWordTokenizer, TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()
before_statements = []
after_statements = []
corrections = []
tokens = set()
for ds in all_ds:
    for example in ds:
        before = example['src_tokens']
        after = example['tgt_tokens']
        correct = example['corrections']
        # for token in before:
        #     if len(token) < 4 and token not in tokens and ("'" in token or "." in token or "-" in token or "-" in token or "*" in token or '£' in token):
        #         tokens.add(token)
        #         print(token)
        #         print(before)
        #         print(detokenize(before))
        before_statements.append(detokenizer.detokenize(before))
        after_statements.append(detokenizer.detokenize(after))
        corrections.append(correct)
tokens = list(tokens)
for token in tokens:
    print(token)

In [7]:
data = {
    "Before Statements": before_statements,
    "After Statements": after_statements,
    "Error Correction": corrections
}
df = pd.DataFrame(data)
df.to_csv("../csv_data/CLC_FCE.csv", index = False)

In [8]:
from datasets import load_dataset
dataset = load_dataset("MohamedAshraf701/lang-8")

In [9]:
ds = dataset['train']

In [10]:
inputs = ds['processed_input']
outputs = ds['processed_output']

In [11]:
import json
output_file = "/Users/frankli/Desktop/Yale Classes/Classes 2024/Ling/final/ling-final-project/llm-finetuning/data/lang8.jsonl"
with open(output_file, 'w', encoding='utf-8') as f:
    for input, output in zip(inputs, outputs):
        example = {"input": input, "output": output, "correction": []}
        f.write(json.dumps(example, ensure_ascii=False) + '\n')
    
    